In [1]:
############-----------author:arioboo------------###############
%load_ext autoreload    
%autoreload 2
#-pip_installs-#
#!pip3 install --upgrade pip
#!pip3 install -r requirements.txt

In [15]:
#-<main_modules>-#
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from glob import glob
import numpy as np
from astropy.io import fits
import random
import sys, os
#-<added>-#
import h5py
import datetime
#import importlib                  #<ME>#could be useful
#-<my_imports>-#
from params.data_params import *
#from tools.saver import write_output_fits

### tensorflow backend uses tf.keras when called keras

In [16]:
print_parameters()


--------------------------------------------------------
VELA_id: 02   || filtro: i
--------------------------------------------------------
parent_dir:   /notebooks/CLUMPS_VELA
python_dir:   /notebooks/CLUMPS_VELA/python_work
sex_dir:      /notebooks/CLUMPS_VELA/sextractor_work
noise_gen_dir:/notebooks/CLUMPS_VELA/noise_gen                           || noise_dir:/notebooks/CLUMPS_VELA/noise_stamps
models_dir:   /notebooks/CLUMPS_VELA/models


VELA_list:    ['VELA02', 'VELA04', 'VELA05', 'VELA06']
absVELA_list: ['/notebooks/CLUMPS_VELA/VELA02', '/notebooks/CLUMPS_VELA/VELA04', '/notebooks/CLUMPS_VELA/VELA05', '/notebooks/CLUMPS_VELA/VELA06']
ID_list:      ['02', '04', '05', '06']
VELA_dirs:    {'06': '/VELA06', '02': '/VELA02', '05': '/VELA05', '04': '/VELA04'}
absVELA_dirs: {'05': '/notebooks/CLUMPS_VELA/VELA05', '02': '/notebooks/CLUMPS_VELA/VELA02', '06': '/notebooks/CLUMPS_VELA/VELA06', '04': '/notebooks/CLUMPS_VELA/VELA04'}


noise_gen_dir: /notebooks/CLUMPS_VELA/noise_gen
sb00_lo

In [17]:
print("###---<START of unet_me.py>---###")

###---<START of unet_me.py>---###


In [18]:

__all__ = ['UNet']

In [19]:
# Read CANDELS images        ###--(CORE)
def read_files_real (input_path, img_size, n = None):

    # first, lets read in all the fits files in the relevant directory
    files = glob.glob(input_path+'/*.fits')
    GalID = np.array([f.split('/')[-1][6:-5] for f in files])
    #GalID = np.array([f.split('/')[-1].split('_')[1] for f in files])     #(OCODE!)   

    if (n is None):
        n = len(files) ; n=int(n)

    # allocate
    imgs = np.empty((n,img_size,img_size))

    print ("Reading in " + str(n) + " fits files...")

    dot_freq = int(np.round(n/100.+0.5))

    for i in range (0,n):
        if (i % dot_freq == 0):
            sys.stdout.write(str(i/dot_freq)+".")
            sys.stdout.flush()
        try:
            hdul = fits.open(files[i])
            imgs[i] = hdul[0].data
        except Exception as excep:
            continue

    return imgs, GalID

In [20]:
def UNet(input_shape, init_filt_size=64):
    """
    U-Net image segmentation model

    Parameters
    ----------
    input_shape : tuple of ints
        Shape of the input images
        The order depends on which keras backend is used.
            - TensorFlow => (row_size, col_size, channels)
            - Theano => (channels, row_size, col_size)
    init_filt_size : int, optional
        Size of the first filter (default is 64)
        It determines automatically the size of the next filters

    Returns
    -------
    model : Keras model

    References
    ----------
    https://arxiv.org/abs/1505.04597v1

    """
    img_input = Input(shape=input_shape)

    nfilt1 = init_filt_size
    nfilt2 = nfilt1 * 2
    nfilt3 = nfilt2 * 2

    # Block 1
    x = Conv2D(nfilt1, (3, 3), activation='relu', padding='same',
               name='block1_conv1')(img_input)
    x_1a = Conv2D(nfilt1, (3, 3), activation='relu', padding='same',
                  name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2),
                     name='block1_pool')(x_1a)

    # Block 2
    x = Conv2D(nfilt2, (3, 3), activation='relu', padding='same',
               name='block2_conv1')(x)
    x_2a = Conv2D(nfilt2, (3, 3), activation='relu', padding='same',
                  name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2),
                     name='block2_pool')(x_2a)

    # Block 3
    x = Conv2D(nfilt3, (3, 3), activation='relu', padding='same',
               name='block3_conv1')(x)
    x = Conv2D(nfilt3, (3, 3), activation='relu', padding='same',
               name='block3_conv2')(x)
    x_2b = Conv2DTranspose(nfilt2, (2, 2), strides=(2, 2),
                           input_shape=(None, 23, 23, 1),
                           name='block3_deconv1')(x)

    # Deconv Block 1
    x = concatenate([x_2a, x_2b])
    x = Conv2D(nfilt2, kernel_size=(3, 3), activation='relu',
               padding='same', name='dblock1_conv1')(x)
    x = Conv2D(nfilt2, (3, 3), activation='relu', padding='same',
               name='dblock1_conv2')(x)
    x_1b = Conv2DTranspose(nfilt1, kernel_size=(2, 2), strides=(2, 2),
                           name='dblock1_deconv')(x)

    # Deconv Block 2
    x = concatenate([x_1a, x_1b], input_shape=(None, 92, 92, None),
                    name='dbock2_concat')
    x = Conv2D(nfilt1, (3, 3), activation='relu', padding='same',
               name='dblock2_conv1')(x)
    x = Conv2D(nfilt1, (3, 3), activation='relu', padding='same',
               name='dblock2_conv2')(x)

    # NOTE: this line hardcodes the number of output channels (currently == 1).
    # Output convolution.
    x = Conv2D(1, (1, 1), activation=None, padding='same',
               name='dblock2_conv3')(x)

    # Create model
    model = Model(img_input, x, name='UNet')

    return model


In [21]:
#parent_dir = '/notebooks/CLUMPS_VELA'    # parent_dir defined ->  from "params.sys_params" import *

#GAL_folder (from "data_params.py")
#img_dir
try:    os.stat(output_folder)    ;print("Statting folder:",output_folder) 
except: os.mkdir(output_folder)   ;print("Making folder:  ",output_folder)
    
#saved_outputs
print('-----------------------------')
print("Prepared_Images directory:   ",img_folder)       ##dl+'/galsim/data/clumps_50000'+date
print("Unet_Output_Images directory:",output_folder)


Statting folder: /notebooks/CLUMPS_VELA/python_work/DATA_outputs/VELA02_F775W
-----------------------------
Prepared_Images directory:    /notebooks/CLUMPS_VELA/python_work/DATA/VELA02_F775W
Unet_Output_Images directory: /notebooks/CLUMPS_VELA/python_work/DATA_outputs/VELA02_F775W


In [22]:
#DATE AND MODEL

#date         #data_params.py  #(D!)                     
#model_name   #data_params.py

model = UNet ((img_size,img_size,1))
model.compile(optimizer = Adam(lr = 1e-3),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


In [23]:
READ_DATA    = 1   # 1
#TRAIN        = 0   # 0
#PREDICT      = 0   # 0
PREDICT_REAL = 1   # 1

In [24]:
if READ_DATA:
    print ("Reading input data...")
    
    imgs, GalID = read_files_real (img_folder, img_size)                                         #(OCODE:) imgs,img_masks,GalID = read_files(img_folder,img_siz)
    np.save(output_folder + '/imgs_' + str(imgs.shape[0]) + date + '.npy', imgs)                       #for "read_files" and "read_files_real"
    #np.save(output_folder + '/img_masks_' + str(img_masks.shape[0]) + date + '.npy', img_masks)       #for "read_files" routine
    #np.save(saved_outputs+'/imgs_'+str(imgs.shape[0])+date+'_IDs.npy', GalID)
    ## no me gusta el "GalID", no está bien definido con mis datos.
    
    print ("Loading images..")
    #imgs = np.load(saved_outputs+'/imgs_49879.npy')
    #imgs = np.load(saved_outputs+'/imgs_50000_05_24_18.npy')
    #imgs = np.load(saved_outputs+'/imgs_50000_07_05_18.npy')
    #imgs = np.load(saved_outputs+'/imgs_50000_09_15_18.npy')
    print('imgs.shape[0]=',imgs.shape[0])
    imgs = np.load(output_folder + '/imgs_' + str(imgs.shape[0]) + date + '.npy')

    '''  #<ME>
    print ("Loading masks...")
    #img_masks = np.load(saved_outputs+'/img_masks_49879.npy')
    #img_masks = np.load(saved_outputs+'/img_masks_50000_05_24_18.npy')
    #img_masks = np.load(saved_outputs+'/img_masks_50000_07_05_18.npy')
    #img_masks = np.load(saved_outputs+'/img_masks_50000_09_15_18.npy')
    
    print('img_masks.shape[0]=',img_masks.shape[0])
    img_masks = np.load(output_folder + '/img_masks_' + str(img_masks.shape[0]) + date + '.npy')
    '''  #</ME>
    
    print ("done.")
    #print('stop1')
    imgs = np.expand_dims(imgs,3)
    #img_masks = np.expand_dims(img_masks,3)   #<ME>

Reading input data...
Reading in 475 fits files...
0.0.1.0.2.0.3.0.4.0.5.0.6.0.7.0.8.0.9.0.10.0.11.0.12.0.13.0.14.0.15.0.16.0.17.0.18.0.19.0.20.0.21.0.22.0.23.0.24.0.25.0.26.0.27.0.28.0.29.0.30.0.31.0.32.0.33.0.34.0.35.0.36.0.37.0.38.0.39.0.40.0.41.0.42.0.43.0.44.0.45.0.46.0.47.0.48.0.49.0.50.0.51.0.52.0.53.0.54.0.55.0.56.0.57.0.58.0.59.0.60.0.61.0.62.0.63.0.64.0.65.0.66.0.67.0.68.0.69.0.70.0.71.0.72.0.73.0.74.0.75.0.76.0.77.0.78.0.79.0.80.0.81.0.82.0.83.0.84.0.85.0.86.0.87.0.88.0.89.0.90.0.91.0.92.0.93.0.94.0.Loading images..
imgs.shape[0]= 475
done.


In [12]:
if (PREDICT_REAL):

    print ("Checking model predictions...")
    
    model.load_weights(   os.path.join(models_dir,model_name+'.hd5')   )
    ##Originally, exceed of data in the same folder
    img_test = imgs[:]              #[max_n:] #original process images
    #img_mask_test = img_masks[:]    #[max_n:] #binary mask of img_test
    img_pred = model.predict(img_test)   #prediction of img_test   #img_mask_pred
    
    (n, m , _, _) = img_test.shape 
    
    test_output = np.zeros((n,m,m,3))                #; print(n,m)               #<ME> create matrix of zeros
    test_output[:,:,:,0] = img_test[:,:,:,0]         ; print(img_test)
    #test_output[:,:,:,1] = img_mask_test[:,:,:,0]    ; print(img_mask_test)     #<ME>
    test_output[:,:,:,1] = img_pred[:,:,:,0]         ; print(img_pred)      #<ME> error!
        
    print ("done.")
    
    print ("Saving test results...")
    
    np.save(output_folder + '/img_test-' + str(img_test.shape[0]) + date + '.npy', test_output)


Checking model predictions...
[[[[ 6.03299355e-04]
   [ 3.59934382e-03]
   [ 3.51587869e-03]
   ...
   [-6.30167197e-04]
   [-2.56513269e-03]
   [-7.45640544e-04]]

  [[ 2.80491798e-03]
   [ 2.18645390e-03]
   [ 1.28489034e-03]
   ...
   [ 4.24122112e-03]
   [-1.89078331e-04]
   [-1.96360913e-03]]

  [[-4.93066385e-04]
   [-1.76736049e-03]
   [-1.23816961e-03]
   ...
   [ 1.61943235e-03]
   [ 4.99851158e-05]
   [ 6.66373177e-04]]

  ...

  [[ 3.09442356e-03]
   [-1.76287605e-03]
   [ 4.71880799e-03]
   ...
   [-1.63711666e-03]
   [ 1.18596468e-03]
   [-5.84084122e-03]]

  [[ 4.82211541e-03]
   [ 7.21942633e-05]
   [-8.18766828e-04]
   ...
   [ 1.30975037e-03]
   [ 6.23664702e-04]
   [-1.82753149e-03]]

  [[-6.35971082e-06]
   [ 1.79548934e-03]
   [ 2.23232503e-03]
   ...
   [ 7.74130225e-03]
   [-1.90194254e-03]
   [ 5.13229379e-03]]]


 [[[ 6.03299355e-04]
   [ 3.59934382e-03]
   [ 3.51587869e-03]
   ...
   [-6.30167197e-04]
   [-2.56513269e-03]
   [-7.45640544e-04]]

  [[ 2.80491798e

In [13]:
###---<ME><Write fits from predicted images>---###
'''
#predict_extension (sys_params)
#write_output_fits(img_folder,output_folder,img_pred,predict_extension)
'''
real_filenames = glob.glob(img_folder + "/*.fits")                                      
filename_str   = [ i.split("/")[-1].split(".fits")[0] for i in real_filenames]                 
for num_file in range(0,len(filename_str)):    
    fits.writeto(os.path.join(output_folder,filename_str[num_file] + predict_extension + ".fits"),
                img_pred[num_file,:,:,0],overwrite=True)   #(I!!)  #predict_extension!! = "-pred"
    #no he considerado meter las cabeceras "header" de los real_files





import matplotlib.pylab as plt
plt.ion()

num_file=200                                 #<ME> CHANGE THIS (I!!)

###---<check real_files>---###
i_filename = filename_str[num_file]      
with fits.open(os.path.join(img_folder , i_filename + ".fits") ) as hdul:
    plt.imshow(hdul[0].data)

###---<check pred_files>---###
#plt.imshow(img_mask_pred[num_file,:,:,0])
pred_filenames = glob.glob(output_folder + "/*-pred.fits")        #;print(pred_filenames)

i_filename_pred = i_filename + "-pred"
with fits.open( os.path.join(output_folder , i_filename_pred+".fits") )  as hdul:
    plt.imshow(hdul[0].data)

###--------------------------------THE_END----------------------------------------###

###---OTHER_COMMANDS---###




In [14]:
print("###---<END of unet_me.py>---###")
###----------------------END------------------------̣###

###---<END of unet_me.py>---###
